Epidemiological Model Assignment - Parameter exploration

Understanding disease dynamics
Overview

Duration: 3-5 hours
Group size: 3-4 students
Format: Jupyter Notebook
Assessment: Repository setup (5%), Code quality (30%), Documentation (25%), Results interpretation (40%)

This assignment focuses on systematic parameter exploration to understand how different factors affect epidemic outcomes.

Setup and submission format
Jupyter notebook structure

Your submission should be a single notebook with the following sections:

    Setup & Imports (provided starter code)
    Part 1: Parameter Analysis Function
    Part 2: Scenario Comparison
    Part 3: Policy Recommendations
    Conclusions


Required Libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint
import seaborn as sns

Assignment task: Recovery rate investigation

Using the SIRD model from your practical as a starting point, you will investigate how the recovery rate affects epidemic outcomes.
Part 1: Parameter analysis function (50 points)

Task: Create a function called analyze_recovery_rates() that systematically explores different recovery rates.
1.1 Function implementation

Create a well-documented function with the following signature:

In [8]:
def sird_model(y, t, beta, gamma, mu):
    S,I,R,D= y
    N = S+I+R+D
    dSdt = -beta * S * I / N
    dIdt = (beta * S * I / N - gamma * I) - mu * I
    dRdt = gamma * I
    dDdt = mu * I
    return dSdt, dIdt, dRdt, dDdt
def run_sird_simulation(beta, gamma, mu, N=1000, I0=10, days=150):
    S0 = N - I0
    R0 = 0
    D0 = 0
    y0 = [S0, I0, R0, D0]

    t = np.linspace(0, days, days)
    sol = odeint(sird_model,  y0, t, args=(beta, gamma, mu))
    S, I, R, D = sol.T
    return t,S,I,R,D


def analyze_recovery_rates(beta, mu, N, I0, simulation_days):
    """
    Analyze epidemic outcomes for different recovery rates.

    Parameters:
    -----------
    beta : float
        Transmission rate
    mu : float  
        Mortality rate
    N : int
        Total population
    I0 : int
        Initial infected individuals
    simulation_days : int
        Simulation duration in days

    Returns:
    --------
    pandas.DataFrame
        Results summary for each recovery rate
    """
    # Your implementation here

    gamma_values = [0.05, 0.1, 0.15, 0.2, 0.25]
    res=[]
    for gamma in gamma_values:
        t,S,I,R,D = run_sird_simulation(beta, gamma, mu, N, I0, simulation_days)
        peak_infected=int(np.max(I))
        peak_day=int(np.argmax(I))
        total_deaths=int(D[-1])
        R0=beta/gamma

        res.append({
            "gamma" :gamma,
            "R0": R0,
            "peak_infected": peak_infected,
            "peak_day": peak_day,
            "total_deaths": total_deaths
            
        })
    return pd.DataFrame(res)



In [9]:
df =analyze_recovery_rates(beta=0.3, mu=0.01, N =1000, I0=10, simulation_days=150 )
print(df)

   gamma   R0  peak_infected  peak_day  total_deaths
0   0.05  6.0            479        25           165
1   0.10  3.0            269        27            83
2   0.15  2.0            136        30            47
3   0.20  1.5             57        33            26
4   0.25  1.2             18        29            11


1.2 Requirements

    Test recovery rates: gamma_values = [0.05, 0.1, 0.15, 0.2, 0.25]
    For each γ value, calculate:
        Peak number of infectious individuals
        Day when peak occurs
        Total deaths at end of simulation
        Basic reproduction number (R₀ = β/γ)
    Return results as a formatted pandas DataFrame
    Generate a publication-quality plot showing all epidemic curves

1.3 Expected output format

Your function should produce:

    A DataFrame with columns: ['gamma', 'R0', 'peak_infected', 'peak_day', 'total_deaths']
    A matplotlib figure with properly labeled axes, legend, and title
